In [4]:
# !pip install psycopg2
# !pip install pandas

  Using cached psycopg2-2.9.6-cp39-cp39-win_amd64.whl (1.2 MB)


You should consider upgrading via the 'C:\Users\thiag\Documents\each\topicos_avancados_db\venv_tsdb\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import psycopg2

In [2]:
# além da documentação, podemos ver um data frame e seus dados para poder criar as tabelas necessárias
df = pd.read_parquet("./yellow_tripdata_2023-01.parquet")
df.shape

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
CONNECTION = "<URL DA CONEXAO AQUI>"
conn = psycopg2.connect(CONNECTION)
cursor = conn.cursor()

In [8]:
# criar a tabela principal de corridas
cursor.execute("""
    CREATE TABLE "rides"(
        vendor_id TEXT,
        pickup_datetime TIMESTAMP WITHOUT TIME ZONE NOT NULL,
        dropoff_datetime TIMESTAMP WITHOUT TIME ZONE NOT NULL,
        passenger_count NUMERIC,
        trip_distance NUMERIC,
        rate_code INTEGER,
        store_and_fwd_flag TEXT,
        pickup_location_id INTEGER,
        dropoff_location_id INTEGER,
        payment_type INTEGER,
        fare_amount NUMERIC,
        extra NUMERIC,
        mta_tax NUMERIC,
        tip_amount NUMERIC,
        tolls_amount NUMERIC,
        improvement_surcharge NUMERIC,
        total_amount NUMERIC,
        congestion_surcharge NUMERIC,
        airport_fee NUMERIC
    );""")

In [9]:
# tornar a tabela principal de corridas uma Hypertable
cursor.execute("SELECT create_hypertable('rides', 'pickup_datetime', 'payment_type', 2, create_default_indexes=>FALSE);")

In [10]:
# definir index na hypertable para facilitar as queries
cursor.execute("""
CREATE INDEX ON rides (vendor_id, pickup_datetime DESC);
CREATE INDEX ON rides (rate_code, pickup_datetime DESC);
CREATE INDEX ON rides (passenger_count, pickup_datetime DESC);""")

In [11]:
# criar a tabela de apoio de tipo Payment_type
cursor.execute("""
CREATE TABLE IF NOT EXISTS "payment_types"(
   payment_type INTEGER,
   description TEXT
);
INSERT INTO payment_types(payment_type, description) VALUES
(1, 'credit card'),
(2, 'cash'),
(3, 'no charge'),
(4, 'dispute'),
(5, 'unknown'),
(6, 'voided trip');
""")

In [12]:
# cria a tabela de apoio da coluna RateCodeID
cursor.execute("""
CREATE TABLE IF NOT EXISTS "rates"(
    rate_code   INTEGER,
    description TEXT
);
INSERT INTO rates(rate_code, description) VALUES
(1, 'standard rate'),
(2, 'JFK'),
(3, 'Newark'),
(4, 'Nassau or Westchester'),
(5, 'negotiated fare'),
(6, 'group ride');
""")

In [13]:
# cria a tabela de apoio da coluna VendorID
cursor.execute("""
CREATE TABLE IF NOT EXISTS "vendor"(
    vendorid   INTEGER,
    description TEXT
);
INSERT INTO rates(rate_code, description) VALUES
(1, 'Creative Mobile Technologies, LLC'),
(2, 'VeriFone Inc.');
""")

In [17]:
conn.commit() # confirma as operações anteriores para tomar efeito no banco de dados 
conn.close() # fecha a conexão com o banco de dados